In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import geopandas as gpd
from shapely import wkt
from shapely.ops import nearest_points, split, snap
from shapely.geometry import Point, LineString, MultiPoint
import folium
import requests
import json
from github import Github

In [39]:

g=Github("ChiaJung-Yeh", "robert13281714")
g.get_user().get_repo('NYCU_TDX')


GithubException: 401 {"message": "Requires authentication", "documentation_url": "https://docs.github.com/rest/reference/users#get-the-authenticated-user"}

In [ ]:
# 下載公共運輸路網圖資
# TDX API
app_id='robert1328.mg10-5fef152e-ee4f-4cfb'
app_key='591fe8aa-6fdc-4c3b-9663-428a62ec8863'

def get_token(app_id, app_key):
    class Auth():
        def __init__(self, app_id, app_key):
            self.app_id = app_id
            self.app_key = app_key

        def get_auth_header(self):
            content_type = 'application/x-www-form-urlencoded'
            grant_type = 'client_credentials'

            return{
                'content-type' : content_type,
                'grant_type' : grant_type,
                'client_id' : self.app_id,
                'client_secret' : self.app_key
            }

    class data():
        def __init__(self, app_id, app_key, auth_response):
            self.app_id = app_id
            self.app_key = app_key
            self.auth_response = auth_response

        def get_data_header(self):
            auth_JSON = json.loads(self.auth_response.text)
            access_token = auth_JSON.get('access_token')

            return{'authorization': 'Bearer '+access_token}
    auth_response=requests.post(auth_url, Auth(app_id, app_key).get_auth_header())
    access_token=data(app_id, app_key, auth_response).get_data_header()
    return(access_token)

In [ ]:
access_token=get_token(app_id, app_key)
county=['Taipei','NewTaipei',"Taoyuan","Taichung","Tainan","Kaohsiung","Keelung","Hsinchu","HsinchuCounty","MiaoliCounty","ChanghuaCounty","NantouCounty","YunlinCounty","ChiayiCounty","Chiayi","PingtungCounty","YilanCounty","HualienCounty","TaitungCounty","KinmenCounty","PenghuCounty","Intercity"]

In [ ]:
def Bus_Route(access_token, county):
    url="https://tdx.transportdata.tw/api/basic/v2/Bus/Route/City/"+county+"?&%24format=JSON"
    data_response=requests.get(url, headers=access_token)
    js_data=json.loads(data_response.text)
    bus_route=pd.DataFrame.from_dict(js_data, orient="columns")
    
    bus_info=bus_route.loc[:,['RouteUID','RouteID','RouteName','BusRouteType','DepartureStopNameZh','DestinationStopNameZh']]
    subroutenum=[len(bus_route.SubRoutes[i]) for i in range(len(bus_route))]
    
    bus_subroute=dict()
    label_all=['SubRouteUID','SubRouteID','SubRouteName','Direction']
    for label_id in label_all:
        if label_id in ['SubRouteName']:
            bus_subroute[label_id]=[bus_route.SubRoutes[i][j][label_id]['Zh_tw'] if label_id in bus_route.SubRoutes[i][j] else None for i in range(len(bus_route)) for j in range(subroutenum[i])]
        else:
            bus_subroute[label_id]=[bus_route.SubRoutes[i][j][label_id] if label_id in bus_route.SubRoutes[i][j] else None for i in range(len(bus_route)) for j in range(subroutenum[i])]

    bus_info=bus_info.iloc[np.repeat(np.arange(len(bus_info)), subroutenum)].reset_index(drop=True)
    bus_route=pd.concat([bus_info, pd.DataFrame(bus_subroute)], axis=1)
    bus_route.RouteName=[bus_route.RouteName[i]['Zh_tw'] for i in range(len(bus_route))]
    return(bus_route)



def Bus_Shape(access_token, county):
    url="https://tdx.transportdata.tw/api/basic/v2/Bus/Shape/City/"+county+"?&%24format=JSON"
    data_response=requests.get(url, headers=access_token)
    js_data=json.loads(data_response.text)
    bus_shape=pd.DataFrame.from_dict(js_data, orient="columns")
    
    bus_shape.RouteName=[bus_shape.RouteName[i]['Zh_tw'] if len(bus_shape.RouteName[i])!=0  else None for i in range(len(bus_shape))]
    bus_shape.SubRouteName=[bus_shape.SubRouteName[i]['Zh_tw'] if len(bus_shape.SubRouteName[i])!=0  else None for i in range(len(bus_shape))]
    bus_shape=bus_shape.loc[:,['RouteUID','RouteID','RouteName','SubRouteUID','SubRouteID','SubRouteName','Geometry']].rename(columns={'Geometry':'geometry'})
    bus_shape['geometry']=bus_shape['geometry'].apply(wkt.loads)
    bus_shape=gpd.GeoDataFrame(bus_shape, crs='epsg:4326')
    return(bus_shape)




In [ ]:
bus_shape=Bus_Shape(access_token, "Taipei")
bus_shape

In [ ]:
#---TDX 介接公車路線---#
county=['Taipei','NewTaipei',"Taoyuan","Taichung","Tainan","Kaohsiung","Keelung","Hsinchu","HsinchuCounty","MiaoliCounty","ChanghuaCounty","NantouCounty","YunlinCounty","ChiayiCounty","Chiayi","PingtungCounty","YilanCounty","HualienCounty","TaitungCounty","KinmenCounty","PenghuCounty","Intercity"]
all_url=[]
for i in county[0:-1]:
    all_url.append("https://tdx.transportdata.tw/api/basic/v2/Bus/Shape/City/"+i+"?&%24format=JSON")
all_url.append("https://tdx.transportdata.tw/api/basic/v2/Bus/Shape/InterCity?&$format=JSON")

bus_shape=pd.DataFrame()
for i in all_url:
    d=data(app_id, app_key, auth_response)
    data_response=requests.get(i, headers=access_token)

    js_data=json.loads(data_response.text)
    bus_shape_temp=pd.DataFrame.from_dict(js_data, orient="columns")
    # pd.DataFrame.from_dict(json.loads(rail_shape_temp.RouteName.to_json()), orient="index")
    bus_shape_temp=bus_shape_temp.loc[:,['RouteUID','Geometry']].rename(columns={'Geometry':'geometry'})
    bus_shape_temp=gpd.GeoDataFrame(pd.concat([bus_shape_temp['RouteUID'], bus_shape_temp['geometry'].apply(wkt.loads)], axis=1), crs='epsg:4326')
    bus_shape=pd.concat([bus_shape, bus_shape_temp]).reset_index(drop=True)
bus_shape=bus_shape.to_crs('EPSG:3826')

county=['Taipei','NewTaipei',"Taoyuan","Taichung","Tainan","Kaohsiung","Keelung","Hsinchu","HsinchuCounty","MiaoliCounty","ChanghuaCounty","NantouCounty","YunlinCounty","ChiayiCounty","Chiayi","PingtungCounty","YilanCounty","HualienCounty","TaitungCounty","KinmenCounty","PenghuCounty","Intercity"]
all_url=[]
for i in county[0:-1]:
    all_url.append("https://tdx.transportdata.tw/api/basic/v2/Bus/Route/City/"+i+"?&%24format=JSON")
all_url.append("https://tdx.transportdata.tw/api/basic/v2/Bus/Route/InterCity?%24format=JSON")

bus_route=pd.DataFrame()
for i in all_url:
    d=data(app_id, app_key, auth_response)
    data_response=requests.get(i, headers=d.get_data_header())

    js_data=json.loads(data_response.text)
    bus_route_temp=pd.DataFrame.from_dict(js_data, orient="columns")

    bus_route_temp=bus_route_temp.loc[:,['RouteUID','BusRouteType']].drop_duplicates().reset_index(drop=True)
    bus_route=pd.concat([bus_route, bus_route_temp]).reset_index(drop=True)

bus_shape=bus_shape.merge(bus_route, on='RouteUID', how='left')
bus_shape=bus_shape[~bus_shape.BusRouteType.isna()].reset_index(drop=True)